In [59]:
import Crypto 
import Crypto.Random
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 
from Crypto.Hash import SHA256
import binascii
import json
from datetime import datetime

In [ ]:
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
    
    def to_dict(self):
        return ({'sender': self.sender, 'recipient': self.recipient, 'value': self.value})
    
    def verify_transaction_signature(self):
        if hasattr(self,'signature'):
            public_key = RSA.importKey(binascii.unhexlify(self.sender))
            verifier = PKCS1_v1_5.new(public_key)
            h = SHA256.new(str(self.to_dict()).ecode('utf-8'))
            return verifier.verify(h,binascii.unhexlify(self.signature))
        else:
            return False
    
    def to_json(self):
        return json.dumps(self._dict_,sort_keys = False)

In [60]:
class Wallet:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024,random)
        self._public_key = self._private_key.publickey()
        
    def sign_transaction(self,transaction: Transaction):
        signer = PKCS1_v1_5.new(self._private_key)
        h = SHA256.new(str(transaction.to_dict()).encode('utf-8'))
        return binascii.hexlify(signer.sign(h).decode('ascii'))
    
    @property
    def identity(self):
        pubkey = binascii.hexlify(self._public_key.exportKey(format="DER"))
        return pubkey.decode('ascii')
    @property
    def private(self):
        privkey = binascii.hexlify(self._private_key.exportKey(format="DER"))
        return privkey.decode('ascii')

In [61]:
Bob = Wallet()
Jimmy = Wallet()
Mary = Wallet()
Tom = Wallet()

t1 = Transaction(Bob.identity, Mary.identity, 550)
signature = Bob.sign_transaction(t1)
t1.add_signature(signature), t1.verify_transaction_signature()

t2 = Transaction(Mary.identity, Jimmy.identity, 100)
signature = Mary.sign_transaction(t2)
t2.add_signature(signature), t2.verify_transaction_signature()

t3 = Transaction(Jimmy.identity, Tom.identity, 1000)
signature = Jimmy.sign_transaction(t3)
t3.add_signature(signature), t3.verify_transaction_signature()

t4 = Transaction(Tom.identity, Jimmy.identity, 300)
signature = Tom.sign_transaction(t4)
t4.add_signature(signature), t4.verify_transaction_signature()

t5 = Transaction(Jimmy.identity, Mary.identity, 50)
signature = Jimmy.sign_transaction(t5)
t5.add_signature(signature), t5.verify_transaction_signature()

Pubkey: 30819f300d06092a864886f70d010101050003818d0030818902818100e3506f7792e30ddeebc805cdd98714a4dc9034c029b06859cb16405e31feed894ee37ad590fae952bcf7df903f222b2290485ce210c5180989fbd1ba88fce0301aa31dca496b2aeb540cda1ff8b2cae7f946dad82d26156b7dc3941e333cd08162286aeee6d4c7e85df45408c54e085c09c124a596c8ec02b1b0864672dcce0d0203010001
Key Length: 324


In [63]:
print('SuwandiRL PrivateKey:',SuwandiRL.private)
print('Key Length:', SuwandiRL.private.__len__())
print("The above program is generated by Suwandi Ryan Loe (student id: 55724681) on", datetime.now())

SuwandiRL PrivateKey: 3082025d02010002818100b66a147c641358599247bbd0e47ab45637ac7dcef2389b01c1d25540921dc125bb2aaec8ef4d4c2a44ceea10c85fd93755127afe8b16b525eb144c5741fd31036887125d706eed73997a0f209ba25b30b10e63225ccf5cb8bad3da8cc591b688dee58afbf22b2c7c59d1869963c8a711b96f8f03c835447a607ad51feac9f05302030100010281800120af5ad40f758a33b9473b9d13844909dadef804ace7f0783fc9c53d69637a534a13d26e0a13158c9468935cd4b446491e18916e98855569252833b0e08537720294d467ac4aebd99ada68d55b54b624fb8e01667212af42cb952ca640b11439dfae4f27b9a0129b4d3989ffff1ff4ebc8fa912379cfd9f139ec4fb8826c41024100bbd9c73e99152de50a4ef153e792c00e8c4f7305f6f25ae9e98d84afd9fa7c997c19396871bbfe92af86cac6aca883980cfe9ae55842d6e548b3b8a054210693024100f8976a5079b1ba4b8b82e383fdd2a40b321ce518a5b6bb0a8e7b9517c9e410e91c9549e36824d7f549d444268d41767a9e8a49b3bda29b09b8a154b01069c7410241009c05251738cdfe705d3f73f0dc45667eaecace1e061d04a582f5a75ee70cc4b036023d4c222aa225f2e36147e1df946c955e62085a662956062c3f0bead403410241008b29b88a7495da049085